In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch

In [2]:
import kagglehub
path = kagglehub.dataset_download("samithsachidanandan/air-quality-data-2019-2025")

Using Colab cache for faster access to the 'air-quality-data-2019-2025' dataset.


In [5]:
# Set device for training (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
data = pd.read_csv(path + "/Air_Quality_Data.csv")

In [4]:
data.head()

,date,city,latitude,longitude,pm25,pm10,o3,no2,so2,co,...,wind_speed_mps,month,day,year,day_of_week,hour,is_weekend,day_of_year,season,AQI_Category
0,2019-01-01,Montgomery,32.3777,-86.3,10.01,20.17,21.14,15.27,6.31,0.42,...,8.2,1,1,2019,1,0,0,1,Winter,Good
1,2019-01-02,Montgomery,32.3777,-86.3,15.09,31.87,38.12,13.91,3.39,0.45,...,4.5,1,2,2019,2,0,0,2,Winter,Moderate
2,2019-01-03,Montgomery,32.3777,-86.3,12.38,29.62,40.72,25.84,5.49,0.57,...,2.3,1,3,2019,3,0,0,3,Winter,Moderate
3,2019-01-04,Montgomery,32.3777,-86.3,9.55,23.36,34.85,19.75,3.30,0.46,...,8.0,1,4,2019,4,0,0,4,Winter,Moderate
4,2019-01-05,Montgomery,32.3777,-86.3,10.27,24.34,30.68,24.84,4.93,0.54,...,4.0,1,5,2019,5,0,1,5,Winter,Moderate


In [33]:
print("Shape of the DataFrame:", data.shape)
print("\nInformation about the DataFrame:")
data.info()
print("\nDescriptive statistics of numerical columns:")
data.describe()
print("\nMissing values per column:")
data.isnull().sum()

Shape of the DataFrame: (122604, 23)

Information about the DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122604 entries, 0 to 122603
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   date              122604 non-null  datetime64[ns]
 1   city              122604 non-null  int64         
 2   latitude          122604 non-null  float64       
 3   longitude         122604 non-null  float64       
 4   pm25              122604 non-null  float64       
 5   pm10              122604 non-null  float64       
 6   o3                122604 non-null  float64       
 7   no2               122604 non-null  float64       
 8   so2               122604 non-null  float64       
 9   co                122604 non-null  float64       
 10  aqi               122604 non-null  int64         
 11  temperature_c     122604 non-null  float64       
 12  humidity_percent  122604 non-null  float64 

,0
date,0
city,0
latitude,0
longitude,0
pm25,0
pm10,0
o3,0
no2,0
so2,0
co,0


In [35]:
data.describe()

,date,city,latitude,longitude,pm25,pm10,o3,no2,so2,co,...,temperature_c,humidity_percent,wind_speed_mps,month,day,year,day_of_week,hour,is_weekend,day_of_year
count,122604,122604.000000,122604.000000,122604.000000,122604.000000,122604.000000,122604.000000,122604.000000,122604.000000,122604.000000,...,122604.000000,122604.000000,122604.000000,122604.000000,122604.000000,122604.000000,122604.000000,122604.0,122604.000000,122604.000000
mean,2022-04-16 12:00:00.000000256,25.000000,39.402698,-93.144659,12.108538,25.233990,30.244954,20.179528,5.047906,0.504318,...,15.123362,59.937690,5.257045,6.301165,15.726290,2021.809068,2.998752,0.0,0.285358,176.405990
min,2019-01-01 00:00:00,0.000000,21.306900,-157.858300,4.600000,10.000000,12.000000,8.000000,2.000000,0.200000,...,2.000000,30.000000,0.500000,1.000000,1.000000,2019.000000,0.000000,0.0,0.000000,1.000000
25%,2020-08-23 18:00:00,12.000000,35.779600,-104.820200,9.600000,20.240000,24.240000,16.190000,4.050000,0.410000,...,10.120000,44.800000,2.900000,3.000000,8.000000,2020.000000,1.000000,0.0,0.000000,86.000000
50%,2022-04-16 12:00:00,25.000000,39.768400,-89.650100,12.110000,25.230000,30.240000,20.180000,5.050000,0.500000,...,15.120000,59.900000,5.300000,6.000000,16.000000,2022.000000,3.000000,0.0,0.000000,172.000000
75%,2023-12-08 06:00:00,38.000000,42.732500,-77.436000,14.620000,30.280000,36.270000,24.200000,6.040000,0.600000,...,20.140000,75.000000,7.600000,9.000000,23.000000,2023.000000,5.000000,0.0,1.000000,266.000000
max,2025-07-31 00:00:00,50.000000,58.301900,-69.779500,19.400000,40.000000,48.000000,32.000000,8.000000,0.800000,...,28.000000,90.000000,10.000000,12.000000,31.000000,2025.000000,6.000000,0.0,1.000000,366.000000
std,NaN,14.719661,5.614355,18.441965,3.345877,6.757962,8.106156,5.406037,1.344879,0.135009,...,6.161575,17.365252,2.738815,3.421219,8.798432,1.909281,1.999280,0.0,0.451586,104.616235


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122604 entries, 0 to 122603
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   date              122604 non-null  object 
 1   city              122604 non-null  object 
 2   latitude          122604 non-null  float64
 3   longitude         122604 non-null  float64
 4   pm25              122604 non-null  float64
 5   pm10              122604 non-null  float64
 6   o3                122604 non-null  float64
 7   no2               122604 non-null  float64
 8   so2               122604 non-null  float64
 9   co                122604 non-null  float64
 10  aqi               122604 non-null  int64  
 11  temperature_c     122604 non-null  float64
 12  humidity_percent  122604 non-null  float64
 13  wind_speed_mps    122604 non-null  float64
 14  month             122604 non-null  int64  
 15  day               122604 non-null  int64  
 16  year              12

In [7]:
def preprocess_train(df):
  df['date'] = pd.to_datetime(df['date'])
  df['year'] = df['date'].dt.year
  df['month'] = df['date'].dt.month
  df['day'] = df['date'].dt.day

  label_encoder = LabelEncoder()
  df['city'] = label_encoder.fit_transform(df['city'])

  return df

In [8]:
data = preprocess_train(data)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122604 entries, 0 to 122603
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   date              122604 non-null  datetime64[ns]
 1   city              122604 non-null  int64         
 2   latitude          122604 non-null  float64       
 3   longitude         122604 non-null  float64       
 4   pm25              122604 non-null  float64       
 5   pm10              122604 non-null  float64       
 6   o3                122604 non-null  float64       
 7   no2               122604 non-null  float64       
 8   so2               122604 non-null  float64       
 9   co                122604 non-null  float64       
 10  aqi               122604 non-null  int64         
 11  temperature_c     122604 non-null  float64       
 12  humidity_percent  122604 non-null  float64       
 13  wind_speed_mps    122604 non-null  float64       
 14  mont

In [10]:
feature_cols = ['city','co','o3','no2','so2','pm10','pm25','aqi']

In [11]:
data_for_aqi_score = data.copy()
data_for_aqi_score = data_for_aqi_score[feature_cols]

In [12]:
data_for_aqi_score

,city,co,o3,no2,so2,pm10,pm25,aqi
0,31,0.42,21.14,15.27,6.31,20.17,10.01,36
1,31,0.45,38.12,13.91,3.39,31.87,15.09,54
2,31,0.57,40.72,25.84,5.49,29.62,12.38,52
3,31,0.46,34.85,19.75,3.30,23.36,9.55,61
4,31,0.54,30.68,24.84,4.93,24.34,10.27,52
...,...,...,...,...,...,...,...,...
122599,50,0.36,20.81,25.58,5.28,27.21,12.49,21
122600,50,0.45,30.74,16.30,5.49,21.95,10.32,43
122601,50,0.49,22.96,25.29,6.60,15.42,16.65,37
122602,50,0.32,23.10,23.18,6.06,28.27,14.15,31


In [13]:
X = data_for_aqi_score.drop('aqi', axis=1)
y = data_for_aqi_score['aqi']

In [14]:
X = X.to_numpy()
y = y.to_numpy()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# seq_model = nn.Sequential(
#     nn.Linear(7, 13),
#     nn.Tanh(),
#     nn.Linear(13, 1))
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=3),
            nn.ReLU()
        )

        # After convs:
        # Input length = 6 (corrected from 7)
        # 6 → 4 → 2
        self.fc = nn.Sequential(
            nn.Linear(32 * 3, 64), # Corrected from 32 * 3 to 32 * 2
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        # x shape: (batch_size, 6)
        x = x.unsqueeze(1)        # → (batch_size, 1, 6)
        x = self.conv(x)          # → (batch_size, 32, 2)
        x = x.view(x.size(0), -1) # flatten: (batch_size, 32 * 2) = (batch_size, 64)
        return self.fc(x)

In [24]:
model = SimpleCNN()
model.to(device) # Move the model to the selected device

SimpleCNN(
  (conv): Sequential(
    (0): Conv1d(1, 16, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (3): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=96, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [25]:
def training_loop(n_epochs, optimiser, model, loss_fn, X_train,  X_val, y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        # Ensure inputs and targets are on the correct device
        X_train_device = X_train.to(device)
        y_train_device = y_train.to(device)
        X_val_device = X_val.to(device)
        y_val_device = y_val.to(device)

        output_train = model(X_train_device) # forwards pass
        loss_train = loss_fn(output_train, y_train_device.unsqueeze(1)) # calculate loss, ensure y_train is 2D
        output_val = model(X_val_device)
        loss_val = loss_fn(output_val, y_val_device.unsqueeze(1)) # ensure y_val is 2D

        optimiser.zero_grad() # set gradients to zero
        loss_train.backward() # backwards pass
        optimiser.step() # update model parameters
        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"+
                  f" Validation loss {loss_val.item():.4f}")

In [19]:
# !pip install -U sympy
# restart runtime

In [20]:
optimiser = torch.optim.SGD(model.parameters(), lr=3e-3)

In [21]:
def to_tensor(data_array):
    tensor = torch.tensor(data_array, dtype=torch.float32)
    if tensor.dim() == 1:
        return tensor.unsqueeze(1)
    return tensor

X_train, X_test, y_train, y_test = map(
    to_tensor, (X_train, X_test, y_train, y_test))

In [26]:
training_loop(
    n_epochs = 100,
    optimiser = optimiser,
    model = model,
    loss_fn = nn.MSELoss(),
    X_train = X_train[0:1000],
    X_val = X_test[0:200],
    y_train = y_train[0:1000],
    y_val = y_test[0:200])

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([1000, 1, 1])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([200, 1, 1])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Training loss 2620.2849, Validation loss 3264.0815
Epoch 10, Training loss 2620.2849, Validation loss 3264.0815
Epoch 20, Training loss 2620.2849, Validation loss 3264.0815
Epoch 30, Training loss 2620.2849, Validation loss 3264.0815
Epoch 40, Training loss 2620.2849, Validation loss 3264.0815
Epoch 50, Training loss 2620.2849, Validation loss 3264.0815
Epoch 60, Training loss 2620.2849, Validation loss 3264.0815
Epoch 70, Training loss 2620.2849, Validation loss 3264.0815
Epoch 80, Training loss 2620.2849, Validation loss 3264.0815
Epoch 90, Training loss 2620.2849, Validation loss 3264.0815
Epoch 100, Training loss 2620.2849, Validation loss 3264.0815


In [27]:
torch.save(model.state_dict(), 'model.pth')

In [28]:
# Instantiate a new model
inference_model = SimpleCNN()

# Load the saved state dictionary
inference_model.load_state_dict(torch.load('model.pth'))

# Move the model to the same device as the input data
inference_model.to(device)

# Set the model to evaluation mode
inference_model.eval()

SimpleCNN(
  (conv): Sequential(
    (0): Conv1d(1, 16, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (3): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=96, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [36]:
# Take the first 5 samples
sample_input_raw = X_test[:5].cpu().numpy()

# Move input to model device
device = next(inference_model.parameters()).device
sample_input_for_model = X_test[:5].to(device)

# Inference
with torch.no_grad():
    predictions = inference_model(sample_input_for_model)

# Actual values
actual_aqi = y_test[:5].cpu().numpy()

print("Sample Input Features (unscaled):\n", sample_input_raw)
print("\nPredicted AQI Scores:\n", predictions.squeeze().cpu().numpy())
print("\nActual AQI Scores:\n", actual_aqi.squeeze())


Sample Input Features (unscaled):
 [[ 7.    0.56 30.13 15.57  4.05 12.03  5.64]
 [46.    0.63 38.93 27.31  6.88 38.87 14.48]
 [19.    0.37 20.7  16.46  4.32 17.34  7.9 ]
 [37.    0.49 40.49 27.6   5.29 28.83 16.73]
 [29.    0.63 30.06 25.66  6.4  18.91  9.38]]

Predicted AQI Scores:
 [-0.2936905  -0.18010934 -0.16295487 -0.3349766  -0.20251824]

Actual AQI Scores:
 [39. 50. 38. 65. 49.]
